In [ ]:
# packets
import time
import ttn
import csv
import os
import numpy as np
import pandas as pd
import math

#initial parameters
total_seats=1
remain_seats=total_seats
temperature=0
count=0
#fill in id you can check it in the TTN website
app_id = "ee5437_project"
access_key = "ttn-account-v2.IXWeU4sMkZ-uXNWEuxDdQhXaRx0DTqEjbZ2fLkNyUF0"
dev_id= "sensor_one"
#sent_message=1
sent_message={ "message": "Try to measure again or go to the hospital" };
#def uplink_callback(msg, client):
  #print("Received uplink from ", msg.dev_id)
  #print(msg)

def downlink_callback(mid, client):
    return 0

def returnTemperature(msg,client):
    global temperature
    global count
    print("app id is:", msg.payload_fields)
    buffer=str(msg.payload_fields)  
    index1=buffer.find('=')
    index2=len(buffer)
    temp=buffer[index1+2:index2-2]
    if temp=='Payment finished':
        count=count-1
    else:
        temperature=float(temp)

    
handler = ttn.HandlerClient(app_id, access_key)


# using mqtt client
while 1 :
    global remain_seats
    mqtt_client = handler.data()
    mqtt_client.set_uplink_callback(returnTemperature)
    mqtt_client.connect()
    if (temperature>37.3):
        con_msg=mqtt_client.send(dev_id, sent_message, port=15,conf=True,sched='replace')
        print(con_msg)
    elif (temperature>=36 and temperature<=37.3):
        count=count+1
        
    remain_seats=total_seats-count      
     
    if (remain_seats<=0):
        sent_full_message={ "message": "Sorry, there is no seat left" };
        mqtt_client.set_downlink_callback(downlink_callback)
        mqtt_client.send(dev_id, sent_full_message, port=15,conf=True)
        count=count-1
    
    temperature=0
    time.sleep(5) # client respond time
    mqtt_client.close()



In [ ]:
import inspect

inspect.signature(mqtt_client.send)